In [1]:
import warnings

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter, FormatStrFormatter

import numpy as np
import pandas as pd
import pathlib
import gym
import gym_woods  # noqa: F401

from lcs import Perception
from lcs.metrics import population_metrics
from myst_nb import glue

from src.diminishing_reward import common_metrics
from src.runner import run_experiments_alternating
from src.observation_wrappers import WoodsBinaryAdapter
from src.decorators import repeat, get_from_cache_or_run
from src.basic_rl import run_q_learning_alternating, run_r_learning_alternating, qlearning, rlearning
from src.utils import build_plots_dir_path, build_cache_dir_path
from src.visualization import diminishing_reward_colors, PLOT_DPI
from src.payoff_landscape import plot_payoff_landscape

from typing import Dict

plt.ioff()  # turn off interactive plotting
plt.style.use('../../../src/phd.mplstyle')

root_dir = pathlib.Path().cwd().parent.parent.parent
cwd_dir = pathlib.Path().cwd()

plot_dir = build_plots_dir_path(root_dir) / cwd_dir.name
cache_dir = build_cache_dir_path(root_dir) / cwd_dir.name


def average_experiment_runs(run_df: pd.DataFrame) -> pd.DataFrame:
    return run_df.groupby(['agent', 'trial', 'phase']).mean().reset_index(level='phase')


def woods_env_provider():
    import gym_woods  # noqa: F401
    return WoodsBinaryAdapter(gym.make(f'Woods1-v0'))

def build_state_perception_map(woods_env):
    result = {}
    state_id = 0

    for raw_state, actions in woods_env.unwrapped._state_action().items():  # actions possible in states
        raw_perception = woods_env.unwrapped._perception(*(map(int, raw_state)))
        result[state_id] = woods_env.observation(raw_perception)
        state_id += 1

    return result

# build a representation of (state_id: perception of a state)
env = woods_env_provider()
state_perception = build_state_perception_map(env)

def get_state_id(perception):
    return list(state_perception.keys())[list(state_perception.values()).index(perception)]

def plot_pop_and_rho(df, plot_filename=None):
    colors = diminishing_reward_colors()

    expl_df = df[df['phase'] == 'exploit']
    xmax = trials/2

    fig, axs = plt.subplots(2, 1, figsize=(18, 16))

    # Steps in trial plot
    for alg in ['ACS2', 'AACS2_v1', 'AACS2_v2', 'Q-Learning', 'R-Learning']:
        alg_df = expl_df.loc[alg]
        idx = pd.Index(name='exploit trial', data=np.arange(1, len(alg_df) + 1))
        alg_df.set_index(idx, inplace=True)

        alg_df['steps_in_trial'].rolling(window=250).mean().plot(ax=axs[0], label=alg, linewidth=2, color=colors[alg])

    axs[0].set_xlabel("Exploit trial")
    axs[0].set_xlim(0, xmax)
    axs[0].xaxis.set_major_locator(MultipleLocator(5000))
    axs[0].xaxis.set_minor_locator(MultipleLocator(1000))
    axs[0].xaxis.set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))
    axs[0].xaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    axs[0].xaxis.set_tick_params(which='minor', size=5, width=1, direction='in')

    axs[0].set_ylabel("Number of steps")
    axs[0].yaxis.set_major_locator(MultipleLocator(5))
    axs[0].yaxis.set_minor_locator(MultipleLocator(1))
    axs[0].yaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    axs[0].yaxis.set_tick_params(which='minor', size=5, width=1, direction='in')

    axs[0].set_title('Steps in trial')
    axs[0].legend(loc='upper right', frameon=False)

    # Rho plot
    for alg in ['AACS2_v1', 'AACS2_v2', 'R-Learning']:
        alg_df = expl_df.loc[alg]
        idx = pd.Index(name='exploit trial', data=np.arange(1, len(alg_df) + 1))
        alg_df.set_index(idx, inplace=True)

        alg_df['rho'].rolling(window=1).mean().plot(ax=axs[1], label=alg, linewidth=2, color=colors[alg])

    axs[1].set_xlabel("Exploit trial")
    axs[1].set_xlim(0, xmax)
    axs[1].xaxis.set_major_locator(MultipleLocator(5000))
    axs[1].xaxis.set_minor_locator(MultipleLocator(1000))
    axs[1].xaxis.set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))
    axs[1].xaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    axs[1].xaxis.set_tick_params(which='minor', size=5, width=1, direction='in')

    axs[1].set_ylabel(r"$\mathregular{\rho}$")
    axs[1].yaxis.set_major_locator(MultipleLocator(100))
    axs[1].yaxis.set_minor_locator(MultipleLocator(25))
    axs[1].yaxis.set_tick_params(which='major', size=10, width=2, direction='in')
    axs[1].yaxis.set_tick_params(which='minor', size=5, width=1, direction='in')
    axs[1].set_ylim(0, 500)

    axs[1].set_title(r'Estimated average $\mathregular{\rho}$')

    fig.tight_layout()

    if plot_filename:
            fig.savefig(plot_filename, dpi=PLOT_DPI, bbox_inches='tight')

    return fig

def plot_population(df, plot_filename=None):
    colors = diminishing_reward_colors()

    expl_df = df[df['phase'] == 'exploit']

    fig, axs = plt.subplots(3, 1, figsize=(18, 16))

    x_lim = int(trials/10)

    for idx, alg in enumerate(['ACS2', 'AACS2_v1', 'AACS2_v2']):
        alg_df = expl_df.loc[alg]
        index = pd.Index(name='exploit trial', data=np.arange(1, len(alg_df) + 1))
        alg_df.set_index(index, inplace=True)

        alg_df['reliable'].rolling(window=50).mean().plot(ax=axs[idx], label='reliable', linewidth=4, color=colors[alg])
        alg_df['population'].rolling(window=50).mean().plot(ax=axs[idx], label='all', linewidth=1, color=colors[alg], alpha=0.8)

        last_rel_count = alg_df['reliable'].iloc[x_lim]
        last_pop_count = alg_df['population'].iloc[x_lim]
        axs[idx].hlines(last_rel_count, 0, x_lim, linestyles='dashed', color=colors[alg], alpha=0.5)
        axs[idx].hlines(last_pop_count, 0, x_lim, linestyles='dashed', color=colors[alg], alpha=0.5)
        axs[idx].text(x_lim/20, last_rel_count-30, fr'${(last_pop_count-last_rel_count):.0f}$ non-reliable classifiers', color=colors[alg])

        axs[idx].set_xlabel("Exploit trial")
        axs[idx].set_xlim(0, x_lim)
        axs[idx].xaxis.set_major_locator(MultipleLocator(1000))
        axs[idx].xaxis.set_minor_locator(MultipleLocator(500))
        axs[idx].xaxis.set_major_formatter(FormatStrFormatter('%1.0f'))
        axs[idx].xaxis.set_tick_params(which='major', size=10, width=2, direction='in')
        axs[idx].xaxis.set_tick_params(which='minor', size=5, width=1, direction='in')

        axs[idx].set_ylabel("Number of classifiers")
        axs[idx].yaxis.set_major_locator(MultipleLocator(100))
        axs[idx].yaxis.set_minor_locator(MultipleLocator(25))
        axs[idx].yaxis.set_tick_params(which='major', size=10, width=2, direction='in')
        axs[idx].yaxis.set_tick_params(which='minor', size=5, width=1, direction='in')

        axs[idx].set_title(f'{alg} population size')
        axs[idx].legend(loc='upper right', frameon=False)

    fig.tight_layout()

    if plot_filename:
        fig.savefig(plot_filename, dpi=PLOT_DPI, bbox_inches='tight')

    return fig

# Experiment 3 - Woods1 environment

In [2]:
learning_rate = 0.8
discount_factor = 0.95
epsilon = 0.8
zeta = 0.0001


# Set ACS2/AACS2 configuration parameter dictionary
basic_cfg = {
    'perception_bits': 24,
    'possible_actions': 8,
    'do_ga': True,
    'beta': learning_rate,
    'epsilon': epsilon,
    'gamma': discount_factor,
    'zeta': zeta,
    'user_metrics_collector_fcn': common_metrics,
    'biased_exploration_prob': 0,
    'metrics_trial_freq': 1
}

NUM_EXPERIMENTS = 4
trials = 50_000


@get_from_cache_or_run(cache_path=f'{cache_dir}/woods1/acs2.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=True)
def run_acs2_in_woods():
    return run_experiments_alternating(woods_env_provider, trials, basic_cfg)


@get_from_cache_or_run(cache_path=f'{cache_dir}/woods1/qlearning.dill')
def run_qlearning_in_woods():
    woods1_env = woods_env_provider()
    init_Q = np.zeros((woods1_env.observation_space.n * 3, woods1_env.action_space.n))
    return run_q_learning_alternating(NUM_EXPERIMENTS, trials, woods1_env, epsilon, learning_rate, discount_factor,
                                      init_Q, perception_to_state_mapper=get_state_id)


@get_from_cache_or_run(cache_path=f'{cache_dir}/woods1/rlearning.dill')
def run_rlearning_in_woods():
    woods1_env = woods_env_provider()
    init_R = np.zeros((woods1_env.observation_space.n * 3, woods1_env.action_space.n))
    return run_r_learning_alternating(NUM_EXPERIMENTS, trials, woods1_env, epsilon, learning_rate, zeta, init_R,
                                      perception_to_state_mapper=get_state_id)


# run computations
acs2_runs_details = run_acs2_in_woods()
q_learning_metrics = run_qlearning_in_woods()
r_learning_metrics = run_rlearning_in_woods()

# average runs and create aggregated metrics data frame
acs2_metrics = [m_df for _, _, _, m_df in acs2_runs_details]

agg_df = pd.concat([
    average_experiment_runs(pd.concat(acs2_metrics)),
    average_experiment_runs(pd.DataFrame(q_learning_metrics)),
    average_experiment_runs(pd.DataFrame(r_learning_metrics))]
)

In [3]:
woods_env = woods_env_provider()

def calculate_state_action_payoffs(state_actions: Dict, pop_acs2, pop_aacs2v1, pop_aacs2v2, Q, R) -> Dict:
    payoffs = {}

    for raw_state, actions in state_actions.items():  # actions possible in states
        raw_perception = woods_env.unwrapped._perception(*(map(int, raw_state)))
        p = Perception(woods_env.observation(raw_perception))
        state_id = get_state_id(p)

        for action in actions:
            # ACS2
            acs2_match_set = pop_acs2.form_match_set(p)
            acs2_action_set = acs2_match_set.form_action_set(action)

            # AACS2_v1
            aacs2v1_match_set = pop_aacs2v1.form_match_set(p)
            aacs2v1_action_set = aacs2v1_match_set.form_action_set(action)

            # AACS2_v2
            aacs2v2_match_set = pop_aacs2v2.form_match_set(p)
            aacs2v2_action_set = aacs2v2_match_set.form_action_set(action)

            # Check if all states are covered
            for alg, action_set in zip(['ACS2', 'AACS2_v1', 'AACS2_v2'],
                                       [acs2_action_set, aacs2v1_action_set,
                                        aacs2v2_action_set]):
                if len(action_set) == 0:
                    warnings.warn(f"No {alg} classifiers for perception: {p}, action: {action}")

            payoffs[(state_id, action)] = {
                'ACS2': np.mean(list(map(lambda cl: cl.r, acs2_action_set))),
                'AACS2_v1': np.mean(list(map(lambda cl: cl.r, aacs2v1_action_set))),
                'AACS2_v2': np.mean(list(map(lambda cl: cl.r, aacs2v2_action_set))),
                'Q-Learning': Q[int(state_id), action],
                'R-Learning': R[int(state_id), action]
            }

    return payoffs

# Take first of each algorithm population pass for presenting payoff landscape
pop_acs2, pop_aacs2v1, pop_aacs2v2, _ = acs2_runs_details[1]

@get_from_cache_or_run(cache_path=f'{cache_dir}/woods1/qlearning-single.dill')
def run_single_qlearning():
    Q_init = np.zeros((woods_env.observation_space.n * 3, woods_env.action_space.n))
    Q, _ = qlearning(woods_env, trials, Q_init, epsilon, learning_rate, discount_factor, perception_to_state_mapper=get_state_id)
    return Q

@get_from_cache_or_run(cache_path=f'{cache_dir}/woods1/rlearning-single.dill')
def run_single_rlearning():
    R_init = np.zeros((woods_env.observation_space.n * 3, woods_env.action_space.n))
    R, rho, _ = rlearning(woods_env, trials, R_init, epsilon, learning_rate, zeta, perception_to_state_mapper=get_state_id)
    return R, rho

Q = run_single_qlearning()
R, rho = run_single_rlearning()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    payoffs = calculate_state_action_payoffs(woods_env.unwrapped._state_action(), pop_acs2, pop_aacs2v1, pop_aacs2v2, Q, R)

WARN: No AACS2_v2 classifiers for perception: 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0, action: 0
WARN: No AACS2_v1 classifiers for perception: 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0, action: 2
WARN: No AACS2_v1 classifiers for perception: 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0, action: 3


In [4]:
woods1_performance_fig = plot_pop_and_rho(agg_df, plot_filename=f'{plot_dir}/woods1-performance.png')
woods1_population_fig = plot_population(agg_df, plot_filename=f'{plot_dir}/woods1-population.png')
woods1_payoff_fig = plot_payoff_landscape(payoffs, rho=rho, rho_text_location={'x': 75, 'y': 450}, plot_filename=f'{plot_dir}/woods1-payoff-landscape.png')

glue('51-woods1-performance-fig', woods1_performance_fig, display=False)
glue('51-woods1-population-fig', woods1_population_fig, display=False)
glue('51-woods1-payoff-fig', woods1_payoff_fig, display=False)

````{tabbed} Performance
```{glue:figure} 51-woods1-performance-fig
:name: "51-woods1-performance-fig"
Performance in Woods-1 environment
```
````

````{tabbed} Payoff
```{glue:figure} 51-woods1-payoff-fig
:name: "51-woods1-payoff-fig"
Woods-1 Payoff Landscape
```
````

````{tabbed} Population
```{glue:figure} 51-woods1-population-fig
:name: "51-woods1-population-fig"
Population of Woods-1 environment
```
````

## Statistical verification

```{admonition} Hypothesis testing
:class: tip
Since Woods1 is not scalable it can be compared to different difficulty. I would skip hypothesis test for this case and treat the whole experiment as a tidbit.
```

## Observations
...